# 接口：从协议到抽象基类

## Python文化中的接口和协议

接口在动态类型语言中是怎么运作的呢?

首先，基本的事实是，Python语言没有interface关键字，而且除了抽象基类，每个类都有接口:
类实现或继承的公开属性(方法或数据属性),包括特殊方法，如`__getitem__`和`__add__`；

按照定义，受保护的属性和私有属性不在接口中，即便“受保护的”属性也只是采用命名约定实现的(单个前导下划线)；
私有属性可以轻松地访问，原因也是如此。

不要觉得把公开数据属性放入对象的接口中不妥，因为如果需要，总能实现读值方法和设值方法，把数据属性变为特性，
使用obj.attr句法的客户代码不会受到影响。vector2d类就是这么做的。

```python
class Vector2d:
    typecode = 'd'
    
    def __init__(self,x,y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i i (self.x, self.y))
```

**接口的实用补充定义**：
  > 对象公开方法的子集，让对象在系统中扮演特定的角色。Python文档中的“文件类对象”或“可迭代对象”就是这个意思。这种说法指的不是特定的类，接口是实现特定角色的方法的集合。一个类可能会实现多个接口，从而让实例扮演多个角色。
  
协议是接口，但不是正式的（只由文档和约定定义），因此协议不能像正式接口那样施加限制。一个类可能只实现部分接口，这是允许的。
有时，某些API只要求“文件类对象”返回字节序列的.read()方法，在特定的上下文中可能需要其他文件件操作方法，也可能不需要。

序列协议是Python最基础的协议之一，即便对象只实现了那个协议最基本的一部分，解释器也会负责任地处理:

## Python喜欢序列

如下所示，`Foo`类，它没有继承abc.Sequence，而且只实现了序列协议的一个方法`__getitem__`；

足够访问元素，迭代和使用in

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f:print(i)

0
10
20


In [4]:
20 in f

True

Python中的协议是鸭子类型的一种极端形式，为了迭代对象，解释器会尝试调用两个不同的方法。

下面的例子，着重强调协议的动态本性:

## 使用猴子补丁在运行时实现协议

之前第一章定义的纸牌类，无法洗牌。

In [5]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [6]:
from random import shuffle
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

这个问题的原因是，shuffle函数要调换集合中的元素的位置，而FrenchDeck只实现了不可变的协议。可变的序列还必须提供`__setitem__`

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [11]:
FrenchDeck.__setitem__ = set_card # 将这个函数直接赋值给__setitem__属性

In [12]:
shuffle(deck)
deck[:5]

[Card(rank='9', suit='clubs'),
 Card(rank='Q', suit='spades'),
 Card(rank='A', suit='clubs'),
 Card(rank='J', suit='clubs'),
 Card(rank='8', suit='diamonds')]

这里的关键是，我们知道FrenckDeck类有一个_cards属性，且它是一个可变序列。我们把函数赋值给特殊方法，从而把他衣服到FrenchDeck类上，这种技术叫做猴子补丁。

如上我们还强调了协议是动态的:random.shuffle函数不关心参数的类型，只要那个对象实现了部分可变序列协议即可。

## 定义抽象基类的子类

import 的时候不会检查抽象方法的实现，在运行时实例化FrenchDeck2类时才会真正检查，因此如果没有正确实现某个抽象方法，Python会抛出TypeError异常。

由于MutableSequence抽象基类需要`__delitem__`和`insert`

In [13]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, value):
        self._cards.insert(position, value)

## 标准库中的抽象基类

### collections.abc 模块中的抽象基类

Python3.4在`collections.abc`模块中定义了16个抽象基类。

`Iterable、Container、Sized`:
各个集合应该继承这三个抽象基类，或者至少实现兼容的协议。
+ Iterable通过 `__iter__`
+ Container通过`in`
+ Sized通过`__len__`


`Sequence、Mapping、Set`：
这三个是主要的不可变集合类型，而且各自都有可变的子类。

`MappingView`：
映射方法`.items()`,`.keys()`，`.values()`返回的对象分别是`ItemsView`,`KeysView`和`ValuesView`

`Callable、Hashable`:
这两个抽象基类与集合没有太大的关系，只不过因为collections.abc是标准库中定义抽象基类的第一个模块，而它们又太重要了，因此才把它们放到collections.abc模块中。

`Iterator`：
注意它是Iterable的子类

### 抽象基类的数字塔

numbers定义的是数字塔，依次继承关系是:
+ Number
+ Complex
+ Real
+ Rational
+ Integral

比如检查一个数是否是整数，可以使用isinstance(x, numbers.Real)检查。

## 定义并使用一个抽象基类



In [14]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
        
    @abc.abstractmethod
    def pick(self):
        """随机删除元素，然后将其返回
        
        如果实例为空，这个方法应该抛出`LookupError`
        """
        
    def loaded(self):
        """如果至少有一个元素，返回True,否则返回False"""
        return bool(self.inspect())
    
    def inspect(self):
        """返回一个有序元组，由当前元素构成"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [15]:
class Fake(Tombola):
    def pick(self):
        return 13
    
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

实例化Fake时报错，因为他没有实现load方法。他的父类将这个方法定义为了抽象方法。

### 抽象基类句法详解

声明抽象基类最简单的方式，是继承abc.ABC或其他抽象基类。

```python
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls,...):
        pass
```

与其他方法描述符一起使用时，abstractmethod应该放在最里层。@abstractmethod和def之间不能有其他装饰器。

### 定义Tombola抽象基类的子类



In [16]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self.items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()
        
# 可以不实现一些方法，直接去继承原先的方法。

In [17]:
import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
        
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))
# 可以不实现一些方法，直接去继承原先的方法。

### Tombola的虚拟子类

白鹅类型的一个基本特性:

即便不继承，也有办法吧一个类注册为抽象基类的虚拟子类。这样做，我们保证注册的类忠实地实现了抽象基类定义的接口，而Python会相信我们，从而不做检查。
如果我们说谎了，那么常规的运行时异常会把我们捕获。

注册虚拟子类的方式是在抽象基类上调用register方法，这么做之后，注册的类会变成抽象基类的虚拟子类，而且isinstance和issubclass等函数都能识别，
但是注册的类不会从抽象基类中继承任何方法或属性。

> 虚拟子类不会继承方法，也不会检查，即使在实例化的时候也不会检查。为了避免运行时错误，需要保证所有所需的方法定义清楚。

In [18]:
from random import randrange

@Tombola.register   # 把TomboList注册为Tombola的虚拟子类
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [19]:
# 检查是否是子类
issubclass(TomboList, Tombola)

True

In [20]:
t = TomboList(range(10))
isinstance(t, Tombola)

True

In [21]:
TomboList.__mro__  # 可以看到没有Tombola

(__main__.TomboList, list, object)

## Python使用register方法


虽然可以将其作为装饰器使用，但是更常见的做法还是把它当做函数使用，用于注册其他地方定义的类。

## 鹅的行为有可能像鸭子

即便不注册，抽象基类也能把一个类识别为虚拟子类。

In [22]:
class Struggle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [24]:
issubclass(Struggle, abc.Sized)

True

这是因为`abc.sized`实现了一个特殊的类方法，名为`__subclasshook__`